# Yelp Data Preparation
Extracting only relevant data for the capstone project instead of using the whole 10GB dataset

The data for this project has been extracted from the Yelp dataset on Kaggle. 
This python notebook has been generated using the Kaggle Kernal and therefore, the data was directly accessed into the kernel. 

To access data locally, follow this article: https://medium.com/@jeff.daniel77/accessing-the-kaggle-com-api-with-jupyter-notebook-on-windows-d6f330bc6953

In [1]:
import gc

import numpy as np
import pandas as pd
import json
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/checkin-df/checkin_df.csv
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json
/kaggle/input/yelp-dataset/Dataset_Agreement.pdf


## Business Data

In [2]:
business_json_path = '../input/yelp-dataset/yelp_academic_dataset_business.json'
business_json = pd.read_json(business_json_path, lines=True)

In [3]:
business_df = pd.DataFrame(business_json)
print(business_df.shape)
business_df.head(2)

(209393, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None


In [4]:
# Considering only open businesses 1 = open, 0 = closed
business_df_open = business_df[business_df['is_open']==1]

In [5]:
# dropping irrelevant columns
drop_columns = ['hours','is_open','attributes']
business_df_open = business_df_open.drop(drop_columns, axis=1)

In [6]:
# exploring the top categories
business_df_open_categories = business_df_open.assign(categories = business_df_open.categories
                         .str.split(', ')).explode('categories')

print("Total categories:", len(business_df_open_categories.categories.value_counts()))
print("Total count:", sum(business_df_open_categories.categories.value_counts()))

business_df_open_categories.categories.value_counts()[:10]

Total categories: 1324
Total count: 714703


Restaurants                  43965
Shopping                     28480
Food                         24844
Home Services                20653
Health & Medical             17626
Beauty & Spas                17293
Local Services               14319
Automotive                   13149
Nightlife                     9818
Event Planning & Services     9500
Name: categories, dtype: int64

## Restaurants Data

In [7]:
# We will only focus on 'Restaurants' category (food, bars seem to be contained within the restaurant categories)
restaurant_df = business_df_open[business_df_open.categories.str.contains('Restaurants',na=False)]
print(restaurant_df.shape)
restaurant_df.head()

(43965, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,"Ethnic Food, Food Trucks, Specialty Food, Impo..."
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,"Restaurants, Cheesesteaks, Poutineries"
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,"Japanese, Fast Food, Food Court, Restaurants"
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,"Persian/Iranian, Turkish, Middle Eastern, Rest..."
33,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,"4550 East Cactus Rd, #KSFC-4",Phoenix,AZ,85032,33.602822,-111.983533,4.0,10,"Food, Pretzels, Bakeries, Fast Food, Restaurants"


In [8]:
# List of restaurants
restaurant_id_set = set(restaurant_df.business_id)
len(restaurant_id_set)

43965

In [9]:
restaurant_df = restaurant_df.rename(columns={'name': 'business_name', 'postal code':'postal_code'})

In [10]:
gc.collect()

97

In [11]:
# Write data into CSV file
restaurant_df.to_csv('./restaurant_df.csv', index=False)

## Reviews data

In [12]:
review_json_path = '../input/yelp-dataset/yelp_academic_dataset_review.json'

reviews =[]
j=0
with open(review_json_path) as f:
    for i, line in enumerate(f):
        temp = json.loads(line)
        
        if temp['business_id'] in restaurant_id_set:
            reviews.append(temp)
            j +=1
        if j >= 1100000:               # setting a limit on number of rows to avoid file size issues       
            break

reviews_df = pd.DataFrame(reviews)
reviews_df.head(2)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
1,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [13]:
gc.collect()

20

In [14]:
reviews_df = reviews_df.rename(columns={'text': 'review_text'})

In [15]:
# List of relevant user_id
reviews_user_id_set = set(reviews_df.user_id)
len(reviews_user_id_set)

540335

In [16]:
# Write data into JSON file
reviews_df.to_json('./reviews_df.json', orient='records', lines=True) # index=False cannot be used with orient='records'

In [17]:
gc.collect()

80

## Tips Data

In [18]:
tip_json_path = '../input/yelp-dataset/yelp_academic_dataset_tip.json'

tips =[]

with open(tip_json_path) as f:
    for i, line in enumerate(f):
        temp = json.loads(line)
        if temp['business_id'] in restaurant_id_set:
            tips.append(temp)

tips_df = pd.DataFrame(tips)

In [19]:
tips_df = tips_df.rename(columns={'text': 'tip_text'})
print(tips_df.shape)
tips_df.head(2)

(695917, 5)


,user_id,business_id,tip_text,date,compliment_count
0,AY-laIws3S7YXNl_f_D6rQ,rDoT-MgxGRiYqCmi0bG10g,Very nice good service good food,2016-07-18 22:03:42,0
1,Ue_7yUlkEbX4AhnYdUfL7g,OHXnDV01gLokiX1ELaQufA,It's a small place. The staff is friendly.,2014-06-06 01:10:34,0


In [20]:
# Write data into CSV file
tips_df.to_csv('./tips_df.csv', index=False)

## Check-in Data

In [21]:
checkin_json_path = '../input/yelp-dataset/yelp_academic_dataset_checkin.json'

checkin =[]

with open(checkin_json_path) as f:
    for i, line in enumerate(f):
        temp = json.loads(line)
        if temp['business_id'] in restaurant_id_set:
            checkin.append(temp)

checkin_df = pd.DataFrame(checkin)

Converting the date array into separate rows for each date

In [22]:
from itertools import chain

checkin_df["date"] = checkin_df["date"].str.split(", ")
dates_list = [x if isinstance(x, list) else [x] for x in checkin_df['date']]

checkin_df = pd.DataFrame({
    'business_id' : checkin_df['business_id'].values.repeat([len(x) for x in dates_list]),
    'date' : list(chain.from_iterable(dates_list))
})

checkin_df["date"] = [d.date() for d in pd.to_datetime(checkin_df['date'])]

checkin_df = checkin_df.groupby('business_id').date.agg(['count','min', 'max']).reset_index()

In [23]:
print(checkin_df.shape)
checkin_df.head()

(43039, 4)


,business_id,count,min,max
0,--1UhMGODdWsrMastO9DZw,7,2016-04-26,2019-03-19
1,--6MefnULPED_I942VcFNA,189,2011-06-04,2019-11-09
2,--9e1ONYQuAa-CB_Rrw7Tw,2942,2010-02-08,2019-11-07
3,--DaPTJW3-tB1vP-PfdTEg,97,2012-06-03,2019-04-13
4,--FBCX-N37CMYDfs790Bnw,552,2010-05-31,2019-11-23


In [24]:
# Write data into CSV file
checkin_df.to_csv('./checkin_df.csv', index=False)

In [25]:
gc.collect()

60

## User Data

In [27]:
user_json_path = '../input/yelp-dataset/yelp_academic_dataset_user.json'

users =[]

with open(user_json_path) as f:
    for i, line in enumerate(f):
        temp = json.loads(line)
        if temp['user_id'] in reviews_user_id_set:
            users.append(temp)

users_df = pd.DataFrame(users)

In [28]:
users_df = users_df.drop('friends', 1)
print(users_df.shape)
users_df.head(2)

(540335, 21)


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,2007-07-06 03:27:11,628,225,227,,14,3.57,...,2,1,0,1,11,15,22,22,10,0
1,FOBRPlBHa3WPHFB5qYDlVg,Michelle,564,2008-04-28 01:29:25,790,316,400,"2008,2009,2010,2011,2012,2013",27,3.84,...,4,5,2,1,33,37,63,63,21,5


In [29]:
# Write data into JSON file
users_df.to_json('./users_df.json', orient='records', lines=True)    # index=False cannot be used when orient='records'